In [5]:
import sys
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from scipy.interpolate import griddata
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import radialProfile, glob, cv2

        
class dataset_draw():
    def __init__(self):
        self.epsilon = 1e-8
        self.number_iter = 1000
        self.y = []
        self.error = []
        self.rgb_weights = [0.2989, 0.5870, 0.1140]
        # self.label_list = ['Real','Xray_DRAGAN','Xray_DRAGAN_SR']    
        self.label_list = ['Real','Maps_DCGAN_50','Maps_DCGAN_500']  
        self.dataset = 'LSUN-church'
        self.N = 179
        self.input_size = 256
        self.folder_list = ['./data/DOTA/DOTA256/*',
                            './fid/maps/DCGAN_SR_256_50_evaluate/*',
                            './fid/maps/DCGAN_SR_256_500_evaluate/*']
        # self.folder_list = ['./data/chest_xray/chest_xray256/*',
        #             './fid/chest_xray/DRAGAN_256_50_evaluate/*',
        #             './fid/chest_xray/DRAGAN_SR_256_50_evaluate/*']
                
    def get_psd1D(self, data=None, folder=None):
        psd1D_total = np.zeros([self.number_iter, self.N])
        psd1D_org_mean = np.zeros(self.N)
        psd1D_org_std = np.zeros(self.N)
        cont = 0
        
        if data is not None:
            for iter, x_ in enumerate(data):
                x_ = np.dot(np.asarray(x_), self.rgb_weights)
                f = np.fft.fft2(x_)
                fshift = np.fft.fftshift(f)
                fshift += self.epsilon
                magnitude_spectrum = 20*np.log(np.abs(fshift))
                psd1D = radialProfile.azimuthalAverage(magnitude_spectrum)
                points = np.linspace(0, self.N, num=psd1D.size) # coordinates of a
                xi = np.linspace(0, self.N, num=self.N) # coordinates for interpolation
                interpolated = griddata(points, psd1D, xi, method='cubic')
                interpolated = (interpolated-np.min(interpolated))/(np.max(interpolated)-np.min(interpolated))
                psd1D_total[cont,:] = interpolated  
                cont+=1
                
                if cont == self.number_iter:
                    break

        elif folder is not None:
            for filename in glob.glob(folder):
                img = cv2.imread(filename, 0)
                f = np.fft.fft2(img)
                fshift = np.fft.fftshift(f)
                fshift += self.epsilon

                magnitude_spectrum = 20*np.log(np.abs(fshift))
                psd1D = radialProfile.azimuthalAverage(magnitude_spectrum)
                psd1D = (psd1D-np.min(psd1D))/(np.max(psd1D)-np.min(psd1D))
                psd1D_total[cont,:] = psd1D  

                cont+=1
                
                if cont == self.number_iter:
                    break

        for x in range(self.N):
            psd1D_org_mean[x] = np.mean(psd1D_total[:,x])
            psd1D_org_std[x] = np.std(psd1D_total[:,x])

        return psd1D_org_mean, psd1D_org_std

    def tv(self, legend=True, y_axis=True):
        
        for f in self.folder_list:
            mean, std = self.get_psd1D(folder=f)
            self.y.append(mean); self.error.append(std)
            
        
        for index, label in enumerate(self.label_list):
            if not index:
                continue
            print(self.label_list[index])
            numerator = 0
            denominator = 0
            for i in range(self.N):
                numerator += abs(self.y[index][i] - self.y[0][i])
                denominator += self.y[0][i]
            print(numerator / denominator)


In [6]:
TV = dataset_draw()
TV.tv()

Maps_DCGAN_50
1.0424060551101801
Maps_DCGAN_500
0.8998737813272291
